# 🧬 Módulo 2: Representación y Visualización Molecular
## Actividad 2.7: Búsqueda y Descarga de Estructuras desde Bases de Datos

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_02_representacion_molecular/07_bases_datos.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Buscar y descargar estructuras desde PubChem
- Acceder a estructuras de proteínas desde el Protein Data Bank (PDB)
- Utilizar APIs para consultas automatizadas
- Descargar estructuras en múltiples formatos
- Realizar búsquedas por nombre, fórmula, similitud estructural
- Extraer información y metadatos de compuestos
- Trabajar con ChEMBL para datos de bioactividad
- Crear datasets personalizados para análisis

---

## 1. Instalación de Dependencias

In [ ]:
# Instalación en Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit biopython requests pandas matplotlib py3Dmol chembl_webresource_client -q

print("✓ Dependencias instaladas")

In [ ]:
# Importar librerías necesarias
import requests
import json
import pandas as pd
import numpy as np
from io import StringIO
import time

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw
import py3Dmol

# Para trabajar con estructuras de proteínas
from Bio import PDB
from Bio.PDB import PDBParser, PDBIO

import matplotlib.pyplot as plt
import seaborn as sns

print("✓ Librerías importadas correctamente")

## 2. Bases de Datos Principales

### 📚 Principales Repositorios Químicos y Biológicos

#### Moléculas Pequeñas:
- **PubChem**: >111 millones de compuestos (NIH/NCBI)
- **ChEMBL**: >2 millones con datos de bioactividad (EMBL-EBI)
- **DrugBank**: Base de datos de fármacos aprobados
- **ZINC**: Compuestos comercialmente disponibles

#### Macromoléculas:
- **PDB**: >200,000 estructuras de proteínas/ácidos nucleicos
- **UniProt**: Secuencias y anotaciones de proteínas
- **AlphaFold DB**: Predicciones estructurales

#### Propiedades y Reacciones:
- **NIST Chemistry WebBook**: Datos termodinámicos
- **Reaxys**: Reacciones químicas (comercial)
- **ChemSpider**: Motor de búsqueda (Royal Society of Chemistry)

## 3. PubChem: Búsqueda y Descarga de Compuestos

### 3.1 Búsqueda por Nombre

In [ ]:
def buscar_pubchem_nombre(nombre):
    """
    Busca un compuesto en PubChem por su nombre y obtiene información básica
    
    API REST de PubChem:
    https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{nombre}/property/{propiedades}/JSON
    """
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    
    # Propiedades a obtener
    propiedades = "MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,InChI,InChIKey,IUPACName"
    
    url = f"{base_url}/compound/name/{nombre}/property/{propiedades}/JSON"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if 'PropertyTable' in data and 'Properties' in data['PropertyTable']:
            compuesto = data['PropertyTable']['Properties'][0]
            return compuesto
        else:
            print(f"No se encontró información para '{nombre}'")
            return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error al consultar PubChem: {e}")
        return None

# Ejemplo: Buscar aspirina
print("Buscando 'aspirin' en PubChem...\n")
aspirina = buscar_pubchem_nombre("aspirin")

if aspirina:
    print(f"CID: {aspirina.get('CID', 'N/A')}")
    print(f"Fórmula: {aspirina.get('MolecularFormula', 'N/A')}")
    print(f"Peso Molecular: {aspirina.get('MolecularWeight', 'N/A')} g/mol")
    print(f"SMILES: {aspirina.get('CanonicalSMILES', 'N/A')}")
    print(f"InChI Key: {aspirina.get('InChIKey', 'N/A')}")
    print(f"IUPAC Name: {aspirina.get('IUPACName', 'N/A')}")

In [ ]:
# Visualizar la molécula descargada
if aspirina and 'CanonicalSMILES' in aspirina:
    mol = Chem.MolFromSmiles(aspirina['CanonicalSMILES'])
    img = Draw.MolToImage(mol, size=(400, 400))
    display(img)
    print(f"\nEstructura de {aspirina.get('IUPACName', 'Aspirina')}")

### 3.2 Descarga de Estructura 3D desde PubChem

In [ ]:
def descargar_estructura_3d_pubchem(nombre, formato='SDF'):
    """
    Descarga la estructura 3D de un compuesto desde PubChem
    
    Formatos disponibles: SDF, JSON, XML, ASNT, etc.
    """
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    url = f"{base_url}/compound/name/{nombre}/record/{formato}/?record_type=3d"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar estructura 3D: {e}")
        return None

# Descargar estructura 3D de ibuprofeno
print("Descargando estructura 3D de ibuprofeno desde PubChem...\n")
sdf_content = descargar_estructura_3d_pubchem("ibuprofen", formato='SDF')

if sdf_content:
    # Leer con RDKit
    mol_3d = Chem.MolFromMolBlock(sdf_content)
    
    if mol_3d:
        print(f"✓ Estructura 3D descargada exitosamente")
        print(f"Átomos: {mol_3d.GetNumAtoms()}")
        print(f"Conformaciones: {mol_3d.GetNumConformers()}")
    else:
        print("Error al procesar el archivo SDF")

In [ ]:
# Visualizar en 3D con py3Dmol
if mol_3d:
    viewer = py3Dmol.view(width=600, height=400)
    viewer.addModel(sdf_content, 'sdf')
    viewer.setStyle({'stick': {}})
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    viewer.show()

### 3.3 Búsqueda por Similitud Estructural

In [ ]:
def buscar_similares_pubchem(smiles, umbral=95, max_resultados=10):
    """
    Busca compuestos similares en PubChem usando similitud de Tanimoto
    
    Parámetros:
    -----------
    smiles : str
        SMILES del compuesto de referencia
    umbral : int
        Umbral de similitud de Tanimoto (0-100)
    max_resultados : int
        Número máximo de resultados
    """
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    
    # Primero, obtener el CID del SMILES
    url_cid = f"{base_url}/compound/smiles/{smiles}/cids/JSON"
    
    try:
        response = requests.get(url_cid, timeout=10)
        data = response.json()
        cid = data['IdentifierList']['CID'][0]
        
        # Buscar similares usando el CID
        url_sim = f"{base_url}/compound/fastsimilarity_2d/cid/{cid}/cids/JSON?Threshold={umbral}&MaxRecords={max_resultados}"
        
        response_sim = requests.get(url_sim, timeout=15)
        data_sim = response_sim.json()
        
        if 'IdentifierList' in data_sim and 'CID' in data_sim['IdentifierList']:
            cids_similares = data_sim['IdentifierList']['CID']
            return cids_similares
        else:
            return []
    
    except Exception as e:
        print(f"Error en búsqueda por similitud: {e}")
        return []

# Ejemplo: Buscar compuestos similares a la aspirina
smiles_aspirina = "CC(=O)Oc1ccccc1C(=O)O"
print(f"Buscando compuestos similares a aspirina (umbral 90%)...\n")

similares = buscar_similares_pubchem(smiles_aspirina, umbral=90, max_resultados=5)

if similares:
    print(f"Se encontraron {len(similares)} compuestos similares:")
    print(f"CIDs: {similares}")
else:
    print("No se encontraron compuestos similares")

### 3.4 Obtener Información Detallada de Múltiples Compuestos

In [ ]:
def obtener_info_cids(cids):
    """
    Obtiene información de múltiples CIDs
    """
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    cids_str = ','.join(map(str, cids))
    
    propiedades = "MolecularFormula,MolecularWeight,CanonicalSMILES,IUPACName"
    url = f"{base_url}/compound/cid/{cids_str}/property/{propiedades}/JSON"
    
    try:
        response = requests.get(url, timeout=15)
        data = response.json()
        
        if 'PropertyTable' in data:
            return data['PropertyTable']['Properties']
        return []
    
    except Exception as e:
        print(f"Error al obtener información: {e}")
        return []

# Obtener información de los similares
if similares:
    info_similares = obtener_info_cids(similares)
    
    if info_similares:
        df_similares = pd.DataFrame(info_similares)
        print("\nCompuestos similares a Aspirina:\n")
        display(df_similares[['CID', 'MolecularFormula', 'MolecularWeight', 'IUPACName']])

In [ ]:
# Visualizar las estructuras similares
if info_similares:
    mols = []
    legends = []
    
    for comp in info_similares[:4]:  # Primeros 4
        mol = Chem.MolFromSmiles(comp['CanonicalSMILES'])
        if mol:
            mols.append(mol)
            legends.append(f"CID: {comp['CID']}\nMW: {comp['MolecularWeight']:.1f}")
    
    if mols:
        img = Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(300, 300),
                                   legends=legends, returnPNG=False)
        display(img)

## 4. Protein Data Bank (PDB): Estructuras de Proteínas

### 4.1 Búsqueda y Descarga desde PDB

In [ ]:
def descargar_estructura_pdb(pdb_id, formato='pdb'):
    """
    Descarga una estructura desde el Protein Data Bank
    
    Parámetros:
    -----------
    pdb_id : str
        Código PDB (4 caracteres, ej: '1HSG')
    formato : str
        'pdb' o 'cif'
    """
    if formato == 'pdb':
        url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    elif formato == 'cif':
        url = f"https://files.rcsb.org/download/{pdb_id}.cif"
    else:
        raise ValueError("Formato debe ser 'pdb' o 'cif'")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        return response.text
    
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar estructura: {e}")
        return None

# Ejemplo: Descargar proteasa VIH-1 con inhibidor (1HSG)
pdb_id = "1HSG"
print(f"Descargando estructura PDB: {pdb_id}\n")

pdb_content = descargar_estructura_pdb(pdb_id)

if pdb_content:
    print(f"✓ Estructura descargada exitosamente")
    print(f"Tamaño del archivo: {len(pdb_content)} bytes")
    print(f"\nPrimeras líneas del archivo PDB:\n")
    print('\n'.join(pdb_content.split('\n')[:10]))

### 4.2 Análisis de Estructura PDB con Biopython

In [ ]:
def analizar_estructura_pdb(pdb_content):
    """
    Analiza una estructura PDB usando Biopython
    """
    parser = PDBParser(QUIET=True)
    
    # Parsear desde string
    from io import StringIO
    structure = parser.get_structure('protein', StringIO(pdb_content))
    
    # Información básica
    info = {
        'Modelos': len(structure),
        'Cadenas': [],
        'Residuos': 0,
        'Átomos': 0,
        'Heteroátomos': 0,
    }
    
    for model in structure:
        for chain in model:
            chain_id = chain.id
            residuos = list(chain.get_residues())
            info['Cadenas'].append(chain_id)
            info['Residuos'] += len(residuos)
    
    # Contar átomos
    for atom in structure.get_atoms():
        if atom.get_parent().id[0] == ' ':  # Residuo estándar
            info['Átomos'] += 1
        else:  # Heteroátomo (ligando, agua, etc.)
            info['Heteroátomos'] += 1
    
    return structure, info

# Analizar la estructura descargada
if pdb_content:
    estructura, info = analizar_estructura_pdb(pdb_content)
    
    print(f"\nAnálisis de {pdb_id}:\n")
    print(f"Modelos: {info['Modelos']}")
    print(f"Cadenas: {', '.join(info['Cadenas'])}")
    print(f"Residuos totales: {info['Residuos']}")
    print(f"Átomos de proteína: {info['Átomos']}")
    print(f"Heteroátomos (ligandos, agua): {info['Heteroátomos']}")

### 4.3 Visualización 3D de Proteínas

In [ ]:
# Visualizar con py3Dmol
if pdb_content:
    viewer = py3Dmol.view(width=800, height=600)
    viewer.addModel(pdb_content, 'pdb')
    
    # Estilo cartoon para proteína
    viewer.setStyle({'cartoon': {'color': 'spectrum'}})
    
    # Ligando en sticks
    viewer.addStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})
    
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    viewer.show()

### 4.4 Búsqueda Avanzada en PDB

In [ ]:
def buscar_pdb_por_termino(termino, max_resultados=10):
    """
    Busca estructuras en PDB por término de búsqueda
    
    Usa la API REST de RCSB PDB
    """
    url = "https://search.rcsb.org/rcsbsearch/v2/query"
    
    query = {
        "query": {
            "type": "terminal",
            "service": "text",
            "parameters": {
                "value": termino
            }
        },
        "return_type": "entry",
        "request_options": {
            "results_content_type": ["experimental"],
            "sort": [{"sort_by": "score", "direction": "desc"}],
            "scoring_strategy": "combined"
        }
    }
    
    try:
        response = requests.post(url, json=query, timeout=15)
        data = response.json()
        
        if 'result_set' in data:
            resultados = data['result_set'][:max_resultados]
            pdb_ids = [r['identifier'] for r in resultados]
            return pdb_ids
        return []
    
    except Exception as e:
        print(f"Error en búsqueda PDB: {e}")
        return []

# Ejemplo: Buscar estructuras de lisozima
print("Buscando estructuras de 'lysozyme' en PDB...\n")
resultados_lisozima = buscar_pdb_por_termino("lysozyme", max_resultados=5)

if resultados_lisozima:
    print(f"Se encontraron estructuras:")
    for pdb_id in resultados_lisozima:
        print(f"  - {pdb_id}")

## 5. ChEMBL: Datos de Bioactividad

### 5.1 Búsqueda de Compuestos en ChEMBL

In [ ]:
from chembl_webresource_client.new_client import new_client

def buscar_chembl_nombre(nombre):
    """
    Busca un compuesto en ChEMBL por nombre
    """
    molecule = new_client.molecule
    
    try:
        resultados = molecule.search(nombre)
        return list(resultados)
    except Exception as e:
        print(f"Error al buscar en ChEMBL: {e}")
        return []

# Buscar aspirina en ChEMBL
print("Buscando 'aspirin' en ChEMBL...\n")
resultados_asp = buscar_chembl_nombre("aspirin")

if resultados_asp:
    print(f"Se encontraron {len(resultados_asp)} resultados\n")
    
    # Mostrar el primero
    comp = resultados_asp[0]
    print(f"ChEMBL ID: {comp.get('molecule_chembl_id', 'N/A')}")
    print(f"Nombre: {comp.get('pref_name', 'N/A')}")
    print(f"Fórmula: {comp.get('molecule_properties', {}).get('full_molformula', 'N/A')}")
    print(f"Peso Molecular: {comp.get('molecule_properties', {}).get('full_mwt', 'N/A')}")
    print(f"SMILES: {comp.get('molecule_structures', {}).get('canonical_smiles', 'N/A')}")

### 5.2 Obtener Datos de Bioactividad

In [ ]:
def obtener_bioactividad_chembl(chembl_id):
    """
    Obtiene datos de bioactividad de un compuesto
    """
    activity = new_client.activity
    
    try:
        resultados = activity.filter(molecule_chembl_id=chembl_id)
        return list(resultados)
    except Exception as e:
        print(f"Error al obtener bioactividad: {e}")
        return []

# Obtener bioactividad de aspirina
if resultados_asp:
    chembl_id = resultados_asp[0]['molecule_chembl_id']
    print(f"\nObteniendo datos de bioactividad para {chembl_id}...\n")
    
    bioactividad = obtener_bioactividad_chembl(chembl_id)
    
    if bioactividad:
        print(f"Se encontraron {len(bioactividad)} registros de bioactividad\n")
        
        # Crear DataFrame
        df_bio = pd.DataFrame(bioactividad)
        
        # Seleccionar columnas relevantes
        cols_interes = ['target_pref_name', 'standard_type', 'standard_value', 
                       'standard_units', 'activity_comment']
        cols_disponibles = [col for col in cols_interes if col in df_bio.columns]
        
        if cols_disponibles:
            print("Primeros 5 registros de bioactividad:\n")
            display(df_bio[cols_disponibles].head())

### 5.3 Buscar Compuestos por Target (Diana Terapéutica)

In [ ]:
def buscar_compuestos_por_target(target_name, max_resultados=10):
    """
    Busca compuestos activos contra un target específico
    """
    target = new_client.target
    activity = new_client.activity
    
    try:
        # Buscar target
        targets = target.search(target_name)
        target_list = list(targets)
        
        if not target_list:
            print(f"No se encontró el target '{target_name}'")
            return []
        
        target_chembl_id = target_list[0]['target_chembl_id']
        print(f"Target encontrado: {target_chembl_id}")
        print(f"Nombre: {target_list[0].get('pref_name', 'N/A')}\n")
        
        # Obtener actividades
        actividades = activity.filter(
            target_chembl_id=target_chembl_id,
            standard_type="IC50",
            standard_relation="="
        ).only(['molecule_chembl_id', 'standard_value', 'standard_units'])
        
        return list(actividades)[:max_resultados]
    
    except Exception as e:
        print(f"Error: {e}")
        return []

# Buscar inhibidores de COX-2
print("Buscando inhibidores de COX-2...\n")
compuestos_cox2 = buscar_compuestos_por_target("COX-2", max_resultados=10)

if compuestos_cox2:
    df_cox2 = pd.DataFrame(compuestos_cox2)
    print(f"\nCompuestos encontrados con actividad IC50:\n")
    display(df_cox2[['molecule_chembl_id', 'standard_value', 'standard_units']].head())

## 6. Creación de Datasets Personalizados

In [ ]:
def crear_dataset_farmacos(nombres_farmacos):
    """
    Crea un dataset con información de múltiples fármacos desde PubChem
    """
    dataset = []
    
    for nombre in nombres_farmacos:
        print(f"Descargando {nombre}...", end=' ')
        info = buscar_pubchem_nombre(nombre)
        
        if info:
            # Calcular descriptores adicionales
            smiles = info.get('CanonicalSMILES', '')
            if smiles:
                mol = Chem.MolFromSmiles(smiles)
                if mol:
                    info['LogP'] = Descriptors.MolLogP(mol)
                    info['TPSA'] = Descriptors.TPSA(mol)
                    info['HBD'] = Descriptors.NumHDonors(mol)
                    info['HBA'] = Descriptors.NumHAcceptors(mol)
                    info['RotBonds'] = Descriptors.NumRotatableBonds(mol)
            
            info['NombreBuscado'] = nombre
            dataset.append(info)
            print("✓")
        else:
            print("✗ No encontrado")
        
        time.sleep(0.5)  # Evitar sobrecarga del servidor
    
    return pd.DataFrame(dataset)

# Crear dataset de antiinflamatorios
antiinflamatorios = [
    'aspirin',
    'ibuprofen',
    'naproxen',
    'diclofenac',
    'celecoxib',
]

print("Creando dataset de antiinflamatorios...\n")
df_antiinf = crear_dataset_farmacos(antiinflamatorios)

print(f"\nDataset creado con {len(df_antiinf)} compuestos\n")

In [ ]:
# Visualizar dataset
columnas_mostrar = ['NombreBuscado', 'CID', 'MolecularFormula', 'MolecularWeight', 
                    'LogP', 'TPSA', 'HBD', 'HBA']
cols_disponibles = [col for col in columnas_mostrar if col in df_antiinf.columns]

if cols_disponibles:
    display(df_antiinf[cols_disponibles])

In [ ]:
# Análisis visual del dataset
if 'LogP' in df_antiinf.columns and 'TPSA' in df_antiinf.columns:
    plt.figure(figsize=(10, 6))
    
    plt.scatter(df_antiinf['LogP'], df_antiinf['TPSA'], 
               s=df_antiinf['MolecularWeight'], 
               alpha=0.6, c=range(len(df_antiinf)), 
               cmap='viridis', edgecolors='black', linewidth=2)
    
    # Añadir nombres
    for _, row in df_antiinf.iterrows():
        plt.annotate(row['NombreBuscado'], 
                    (row['LogP'], row['TPSA']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, fontweight='bold')
    
    # Líneas de referencia Lipinski
    plt.axvline(5, color='red', linestyle='--', alpha=0.5, label='LogP = 5')
    plt.axhline(140, color='red', linestyle='--', alpha=0.5, label='TPSA = 140')
    
    plt.xlabel('LogP (Lipofilia)', fontsize=12, fontweight='bold')
    plt.ylabel('TPSA (Å²)', fontsize=12, fontweight='bold')
    plt.title('Espacio Químico de Antiinflamatorios\n(Tamaño = Peso Molecular)', 
             fontsize=14, fontweight='bold', pad=15)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizar estructuras del dataset
if 'CanonicalSMILES' in df_antiinf.columns:
    mols = []
    legends = []
    
    for _, row in df_antiinf.iterrows():
        mol = Chem.MolFromSmiles(row['CanonicalSMILES'])
        if mol:
            mols.append(mol)
            legends.append(f"{row['NombreBuscado']}\nMW: {row['MolecularWeight']:.1f}")
    
    if mols:
        img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(300, 300),
                                   legends=legends, returnPNG=False)
        display(img)

## 7. Guardar y Cargar Datasets

In [ ]:
# Guardar dataset como CSV
if not df_antiinf.empty:
    filename = 'antiinflamatorios_dataset.csv'
    df_antiinf.to_csv(filename, index=False)
    print(f"✓ Dataset guardado como '{filename}'")
    
    # También en formato SDF
    writer = Chem.SDWriter('antiinflamatorios.sdf')
    for _, row in df_antiinf.iterrows():
        if 'CanonicalSMILES' in row:
            mol = Chem.MolFromSmiles(row['CanonicalSMILES'])
            if mol:
                # Añadir propiedades
                mol.SetProp('_Name', str(row['NombreBuscado']))
                mol.SetProp('CID', str(row['CID']))
                mol.SetProp('MolecularWeight', str(row['MolecularWeight']))
                writer.write(mol)
    
    writer.close()
    print(f"✓ Estructuras guardadas como 'antiinflamatorios.sdf'")

## 8. Ejercicios Prácticos

### Ejercicio 1: Dataset de Antibióticos

Crea un dataset de antibióticos betalactámicos.

**Tarea:**
1. Busca en PubChem: penicillin, amoxicillin, cephalexin, ceftriaxone
2. Descarga información y estructuras
3. Calcula descriptores (MW, LogP, HBD, HBA, TPSA)
4. Evalúa la Regla de Lipinski
5. Crea un gráfico LogP vs TPSA
6. Guarda el dataset en formato CSV y SDF

In [ ]:
# TU CÓDIGO AQUÍ

antibioticos = ['penicillin', 'amoxicillin', 'cephalexin', 'ceftriaxone']

# Tu análisis aquí

### Ejercicio 2: Exploración del PDB

Explora estructuras de hemoglobina en el PDB.

**Tarea:**
1. Busca estructuras de "hemoglobin" en el PDB
2. Descarga 2-3 estructuras diferentes
3. Analiza cada una: número de cadenas, residuos, átomos
4. Identifica ligandos presentes
5. Visualiza las estructuras en 3D
6. Compara las estructuras entre sí

In [ ]:
# TU CÓDIGO AQUÍ

# Tu análisis aquí

### Ejercicio 3: Búsqueda por Similitud

Encuentra análogos de un fármaco de interés.

**Tarea:**
1. Elige un fármaco (ej: paracetamol)
2. Busca 10 compuestos similares en PubChem (umbral 85%)
3. Obtén información de todos los similares
4. Calcula descriptores para el conjunto
5. Crea una matriz de similitud entre todos
6. Visualiza las estructuras
7. Identifica cuáles son fármacos conocidos

In [ ]:
# TU CÓDIGO AQUÍ

# Tu análisis aquí

### Ejercicio 4: Análisis de Bioactividad en ChEMBL

Analiza compuestos activos contra una diana terapéutica.

**Tarea:**
1. Busca compuestos activos contra "acetylcholinesterase"
2. Filtra por IC50 < 1000 nM
3. Obtén estructuras de los 10 más potentes
4. Calcula descriptores moleculares
5. Analiza correlaciones entre descriptores y potencia
6. Visualiza el espacio químico (PCA o gráfico 2D)
7. Identifica patrones estructurales comunes

In [ ]:
# TU CÓDIGO AQUÍ

# Tu análisis aquí

### Ejercicio 5: Comparación Multi-Base de Datos

Compara la información del mismo compuesto en diferentes bases de datos.

**Tarea:**
1. Elige un fármaco (ej: "atorvastatin")
2. Búscalo en PubChem y ChEMBL
3. Compara: SMILES, peso molecular, LogP
4. Obtén datos de bioactividad de ChEMBL
5. Busca estructuras 3D en ambas bases
6. Identifica diferencias en la información disponible
7. Crea una tabla comparativa completa

In [ ]:
# TU CÓDIGO AQUÍ

farmaco = "atorvastatin"

# Tu análisis aquí

## Resumen y Conclusiones

En esta actividad has aprendido a:

✅ **Buscar compuestos en PubChem** por nombre, CID o similitud estructural  
✅ **Descargar estructuras 2D y 3D** en múltiples formatos  
✅ **Acceder al Protein Data Bank** para estructuras de macromoléculas  
✅ **Utilizar APIs REST** para consultas automatizadas  
✅ **Trabajar con ChEMBL** para datos de bioactividad  
✅ **Crear datasets personalizados** combinando múltiples fuentes  
✅ **Analizar y visualizar** estructuras descargadas  
✅ **Guardar y gestionar** colecciones de compuestos  

### Puntos Clave

- Las bases de datos químicas contienen millones de estructuras accesibles libremente
- Las APIs permiten automatizar búsquedas y análisis a gran escala
- PubChem es ideal para compuestos pequeños y propiedades fisicoquímicas
- PDB es el repositorio estándar para estructuras de proteínas
- ChEMBL vincula estructura química con bioactividad experimental
- La búsqueda por similitud permite encontrar análogos estructurales
- Es crucial respetar los límites de uso de las APIs

### Mejores Prácticas

1. **Rate limiting**: Añade delays entre consultas masivas
2. **Caché local**: Guarda resultados para evitar consultas repetidas
3. **Validación**: Verifica la calidad de estructuras descargadas
4. **Metadatos**: Registra fuente y fecha de descarga
5. **Formatos estándar**: Usa SDF/MOL para moléculas, PDB/mmCIF para proteínas

### Aplicaciones en Research

- **Virtual Screening**: Construir bibliotecas de compuestos
- **SAR Analysis**: Estudiar relaciones estructura-actividad
- **Target Fishing**: Identificar proteínas relacionadas
- **Benchmarking**: Validar modelos predictivos
- **Data Mining**: Extraer patrones de grandes datasets

### Recursos Adicionales

- [PubChem PUG REST](https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest)
- [RCSB PDB API](https://www.rcsb.org/docs/programmatic-access)
- [ChEMBL Web Services](https://www.ebi.ac.uk/chembl/ws)
- [RDKit Documentation](https://www.rdkit.org/docs/)
- [Biopython Tutorial](https://biopython.org/wiki/Documentation)

### Próximos Pasos

¡Has completado el Módulo 2! Ahora estás listo para:

- **Módulo 3:** Mecánica Molecular y campos de fuerza
- **Módulo 4:** Dinámica Molecular
- **Módulo 5:** Mecánica Cuántica aplicada

---

## Referencias

1. Kim, S. et al. (2021). *PubChem in 2021: new data content and improved web interfaces.* Nucleic Acids Res., 49(D1), D1388-D1395.

2. Berman, H. M. et al. (2000). *The Protein Data Bank.* Nucleic Acids Res., 28(1), 235-242.

3. Gaulton, A. et al. (2017). *The ChEMBL database in 2017.* Nucleic Acids Res., 45(D1), D945-D954.

4. Cock, P. J. et al. (2009). *Biopython: freely available Python tools for computational molecular biology and bioinformatics.* Bioinformatics, 25(11), 1422-1423.

5. PubChem REST API Documentation: https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest

6. RCSB PDB Web Services: https://www.rcsb.org/docs/programmatic-access/web-services-overview

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 2.7: Búsqueda y Descarga de Estructuras desde Bases de Datos**

### 🏆 ¡Has finalizado el Módulo 2!

**Siguiente módulo**: Mecánica Molecular

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_2.6-Descriptores-blue.svg)](06_descriptores.ipynb)
[![Módulo 3](https://img.shields.io/badge/Módulo_3_➡️-Mecánica_Molecular-green.svg)](../modulo_03_mecanica_molecular/README.md)

---

📚 **[Volver al Módulo 2](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>